In [4]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2
from pathlib import Path
home = str(Path.home())

import sys
sys.path = sys.path + [f'{home}/.conda/envs/norm_env/lib/python37.zip', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/lib-dynload', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/site-packages',
                      '../src']
sys.prefix = '/home/joaom/.conda/envs/norm_env'

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import yaml
import matplotlib.pyplot as plt 
import datetime

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import yaml
from jinja2 import Template
cf.themes.THEMES['custom'] = yaml.load(open('cufflinks_template.yaml', 'r'))

In [5]:
from src import utils
from datetime import datetime
conn = utils.connect_athena(path='../configs/athena.yaml')

In [3]:
from tqdm import tqdm_notebook

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
query = """
select *
from spd_sdv_waze_corona.allhist_grid_grid
where region_slug = '{}'
and year = {}
and month = {}
"""

raw_path = Path('/home/joaom/shared/spd-sdv-omitnik-waze/corona/outputs/')

In [ ]:
combinations = pd.read_sql_query("""
select distinct region_slug, year, month, day
from spd_sdv_waze_corona.allhist_grid_grid
""", conn)

In [ ]:
combinations['date'] = combinations.apply(lambda x: pd.Timestamp(year=x['year'], month=x['month'], day=x['day']),1)

In [ ]:
combinations.groupby('region_slug').agg(['max', 'min'])

In [ ]:
for r in combinations['region_slug'].unique():
    (raw_path / r).mkdir(exist_ok=True)

In [ ]:
list(combinations.to_records(index=False))

In [ ]:
for region_slug, year, month in tqdm_notebook(list(combinations.drop_duplicates(subset=['region_slug', 'year', 'month'])
                                                   [['region_slug', 'year', 'month']]
                                                   .to_records(index=False))):
    
    if year == 2020:
        print(query.format(region_slug, year, month))
        df = pd.read_sql_query(query.format(region_slug, year, month), conn)
        
        for day in tqdm_notebook(list(combinations.query(f'year == "{year}"')
                                      .query(f'month == "{month}"')['day'].unique())):
            
    
            to_save = raw_path / region_slug / ('-'.join([str(x) for x in [year, month, day]]) + '.csv')
        
            print(to_save)

            df.query('day == "{day}"').to_csv(to_save, index=False, sep='|')

## Export tables for visualization testing 

Last update 2020-OCT-07

In [ ]:
raw_path = Path('/home/soniame/shared/spd-sdv-omitnik-waze/corona/outputs')

In [11]:
query = """
select *
from spd_sdv_waze_corona.dev_daily_hourly_index
"""
df = pd.read_sql_query(query, conn)
print(df.shape)
df.head()
df.to_csv(f"{raw_path}/sample_data/dev_daily_hourly_index.csv", index = False)

(2904, 7)


In [ ]:
query = """
select *
from spd_sdv_waze_corona.dev_grid_grid_hourly_index
"""
df = pd.read_sql_query(query, conn)
print(df.shape)
df.head()
df.to_csv(f"{raw_path}/sample_data/dev_grid_grid_hourly_index.csv", index = False)